<a href="https://colab.research.google.com/github/karanidenis/transfer_learning_assignment/blob/main/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crop Classification

This project is aimed to use pretrained models with a new dataset to classify different from their images.

## Libraries

In [1]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
train_data_dir = '/content/drive/MyDrive/Colab Notebooks/kag2'

### Data Augmentation Configuration and Training Data Generator Initialization

In [3]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, horizontal_flip=True,vertical_flip=True, rotation_range=90)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=True)

Found 804 images belonging to 5 classes.


In [4]:
train_generator.class_indices

{'jute': 0, 'maize': 1, 'rice': 2, 'sugarcane': 3, 'wheat': 4}

### Libraries for Model Training

In [5]:
from tensorflow.keras.applications import VGG16, InceptionV3, VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

## Pretrained Models

In [6]:
# Load pre-trained models
base_models = [VGG16, InceptionV3, VGG19]
pre_models = []

Initialization of Pre-trained Models and Freezing Layers

In [7]:
for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    pre_models.append(base)

for pretrained_model in pre_models:
    for layer in pretrained_model.layers:
        layer.trainable = False

80134624/80134624 [==============================] - 1s 0us/step


Adding Flatten and Final Dense Layer to Pre-trained Models

In [8]:
models = []
for pretrained_model in pre_models:
    last_layer = Flatten()(pretrained_model.output)
    final_layer = Dense(5, activation='softmax')(last_layer)

    model = Model(inputs=pretrained_model.input, outputs=final_layer)
    models.append(model)

## Compile models

In [9]:
for model in models:
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')

### Summary

In [10]:
for model in models:
    model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### Training

In [12]:
# for model in models:
models[0].fit_generator(train_generator, epochs=5)

<ipython-input-12-933e0d6d48e6>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[0].fit_generator(train_generator, epochs=5)


Epoch 1/5
13/13 [==============================] - 521s 40s/step - loss: 0.3463 - accuracy: 0.8843
Epoch 2/5
13/13 [==============================] - 529s 40s/step - loss: 0.3092 - accuracy: 0.9167
Epoch 3/5
13/13 [==============================] - 526s 40s/step - loss: 0.2821 - accuracy: 0.9266
Epoch 4/5
13/13 [==============================] - 525s 40s/step - loss: 0.2500 - accuracy: 0.9490
Epoch 5/5
13/13 [==============================] - 525s 42s/step - loss: 0.2454 - accuracy: 0.9490


In [14]:
models[1].fit_generator(train_generator, epochs=5)

<ipython-input-14-c27e7f23d13a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[1].fit_generator(train_generator, epochs=5)


Epoch 1/5
13/13 [==============================] - 116s 9s/step - loss: 0.2333 - accuracy: 0.9167
Epoch 2/5
13/13 [==============================] - 113s 9s/step - loss: 0.3930 - accuracy: 0.8918
Epoch 3/5
13/13 [==============================] - 114s 9s/step - loss: 0.3410 - accuracy: 0.8868
Epoch 4/5
13/13 [==============================] - 113s 9s/step - loss: 0.3118 - accuracy: 0.9042
Epoch 5/5
13/13 [==============================] - 123s 9s/step - loss: 0.2194 - accuracy: 0.9241


In [16]:
models[2].fit_generator(train_generator, epochs=5)

<ipython-input-16-abc8b00f14b9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  models[2].fit_generator(train_generator, epochs=5)


Epoch 1/5
13/13 [==============================] - 652s 50s/step - loss: 1.1701 - accuracy: 0.5299
Epoch 2/5
13/13 [==============================] - 646s 50s/step - loss: 1.0274 - accuracy: 0.6057
Epoch 3/5
13/13 [==============================] - 645s 49s/step - loss: 0.8969 - accuracy: 0.6580
Epoch 4/5
13/13 [==============================] - 646s 49s/step - loss: 0.7248 - accuracy: 0.7189
Epoch 5/5
13/13 [==============================] - 647s 50s/step - loss: 0.6582 - accuracy: 0.7600


### Models Evaluation

In [21]:
#TestData
test_data_dir = '/content/drive/MyDrive/Colab Notebooks/crop_images'
test_datagen = ImageDataGenerator(rescale=1./255)

test_batchsize = 64

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224,224),
        batch_size=64,
        class_mode='categorical',
        shuffle=False)


Found 201 images belonging to 5 classes.


In [22]:
evaluations = []
for model in models:
  evaluations.append(model.evaluate_generator(test_generator))


<ipython-input-22-48f512dded22>:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  evaluations.append(model.evaluate_generator(test_generator))


In [23]:
# Print evaluation results
for i, result in enumerate(evaluations):
    print(f'Model {i+1} - Loss: {result[0]}, Accuracy: {result[1]}')

Model 1 - Loss: 0.3885972797870636, Accuracy: 0.8905472755432129
Model 2 - Loss: 0.3878234922885895, Accuracy: 0.9004974961280823
Model 3 - Loss: 0.7806718349456787, Accuracy: 0.6915422677993774
